In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [7]:
splits, info = tfds.load("horses_or_humans",
                         as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'],
                         data_dir="./data")

train_examples, validation_examples, test_examples = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes
num_examples, num_classes

(1027, 2)

In [8]:
BATCH_SIZE = 32
IMAGE_SIZE = 224

In [26]:
# Create a autograph pre-processing function to resize and normalize an image
### START CODE HERE ###
@tf.function
def map_fn(image, label):
  img = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
  img /= 255.
  return img, label

In [27]:
## TEST CODE:

test_image, test_label = list(train_examples)[0]

test_result = map_fn(test_image, test_label)

print(test_result[0].shape)
print(test_result[1].shape)

del test_image, test_label, test_result

(224, 224, 3)
()


In [28]:
train_ds = train_examples.map(map_fn).shuffle(buffer_size=1024).batch(BATCH_SIZE)
valid_ds = validation_examples.map(map_fn).batch(BATCH_SIZE)
test_ds = test_examples.map(map_fn).batch(BATCH_SIZE)